# Data Cleaning

In [1]:
# Install prerequisites

# pip install kaggle

In [2]:
# Import required packages

import json
import os
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Connect to Kaggle API

if "kaggle.json" in os.listdir():
    with open("kaggle.json") as f:
        line = f.readline()
        kaggle_auth = json.loads(line)
        f.close()
else:
    kaggle_auth = {
        "username": "tylerpotts",
        "key": "153fcab7c5aa2b3082e66046bf148821"
    }
os.environ["KAGGLE_USERNAME"] = kaggle_auth["username"]
os.environ["KAGGLE_KEY"] = kaggle_auth["key"]

kaggle_auth

{'username': 'tylerpotts', 'key': 'b4ff2e44f1a3fced8c1657c964be3446'}

In [4]:
# Get Kaggle datasets

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
dataset_names = [
    "osmi/mental-health-in-tech-2016",
    "osmihelp/osmi-mental-health-in-tech-survey-2017",
    "osmihelp/osmi-mental-health-in-tech-survey-2018",
    "osmihelp/osmi-mental-health-in-tech-survey-2019",
]
for dataset_name in dataset_names:
    api.dataset_download_files(dataset_name, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/osmi/mental-health-in-tech-2016
Dataset URL: https://www.kaggle.com/datasets/osmihelp/osmi-mental-health-in-tech-survey-2017
Dataset URL: https://www.kaggle.com/datasets/osmihelp/osmi-mental-health-in-tech-survey-2018
Dataset URL: https://www.kaggle.com/datasets/osmihelp/osmi-mental-health-in-tech-survey-2019


In [5]:
# Upload csvs, organize by year

csv_names = [file for file in os.listdir() if ".csv" in file and file != "data.csv"]
def extract_year(filename):
    pattern = r'\d{4}'
    match = re.search(pattern, filename)
    if match:
        return int(match.group())
    else:
        return 0  # Return 0 if no year is found
csv_names = sorted(csv_names, key=extract_year)
csv_names

['mental-heath-in-tech-2016_20161114.csv',
 'OSMI Mental Health in Tech Survey 2017.csv',
 'OSMI Mental Health in Tech Survey 2018.csv',
 'OSMI 2019 Mental Health in Tech Survey Results - OSMI Mental Health in Tech Survey 2019.csv']

In [6]:
# Turn CSVs into dataframe, mark the year, merge

dfs = []
for csv_name in csv_names:
    filepath = os.path.join(os.getcwd(), csv_name)
    df = pd.read_csv(filepath)

    # Clean dataframe

    if "2016" in csv_name:
        year = 2016
    elif "2017" in csv_name:
        year = 2017
    elif "2018" in csv_name:
        year = 2018
    elif "2019" in csv_name:
        year = 2019
    df["Year"] = [year] * len(df)

    column_name1 = "How many employees does your company or organization have?"

    column_name2 = "What is your age?"
    for i in range(len(df[column_name2].values)):
        value = df[column_name2].values[i]
    try:
        value = int(float(value))
    except:
        value = 0
    if value > 130: # People aren't older than 130
        value = 0
    elif value < 0: # People aren't younger than 0
        value = 0
    df[column_name2].iloc[i] = value

    column_name3 = "What is your gender?"
    for i in range(len(df[column_name3].values)):
        value = df[column_name3].values[i]
        value = str(value).lower().strip()
    if value == "m":
        value = "male"
    elif value == "f":
        value = "female"
    elif value == "dude":
        value = "male"
    elif "female" in value:
        value = "female"
    elif "femail" in value:
        value = "female"
    elif "male" in value:
        value = "male"
    elif "mail" in value:
        value = "male"
    elif "woman" in value:
        value = "female"
    elif "man" in value:
        value = "male"
    elif "boy" in value:
        value = "male"
    elif "girl" in value:
        value = "female"
    else:
        value = "other"
    df[column_name3].iloc[i] = value

    column_name4 = "Do you currently have a mental health disorder?"
    column_name4b = "Do you *currently* have a mental health disorder?"
    if column_name4b in df.columns:
        df = df.rename(columns={column_name4b: column_name4})
    for i in range(len(df[column_name4].values)):
        value = df[column_name4].values[i]
        value = str(value).lower().strip()
    if value in ["maybe", "possibly", "don't know"]:
        value = "not sure"
    df[column_name4].iloc[i] = value

    column_name5 = "What country do you live in?"
    column_name5b = "What country do you <strong>live</strong> in?"
    column_name5c = "What country do you *live* in?"
    if column_name5b in df.columns:
        df = df.rename(columns={column_name4b: column_name5})
    if column_name5c in df.columns:
        df = df.rename(columns={column_name5c: column_name5})

    column_names = [column_name1, column_name2, column_name3, column_name4, column_name5, "Year"]
    data = df.to_dict("records")
    df = pd.DataFrame(data=data, columns=column_names)

    dfs.append(df)

# Merge

df = pd.concat(dfs, ignore_index=True)

# Save as CSV

df.to_csv("data.csv")

In [7]:
# Read CSV

csv = pd.read_csv("data.csv", index_col=0)

csv

,How many employees does your company or organization have?,What is your age?,What is your gender?,Do you currently have a mental health disorder?,What country do you live in?,Year
0,26-100,39.0,Male,No,United Kingdom,2016
1,6-25,29.0,male,Yes,United States of America,2016
2,6-25,38.0,Male,No,United Kingdom,2016
3,NaN,43.0,male,Yes,United Kingdom,2016
4,6-25,43.0,Female,Yes,United States of America,2016
...,...,...,...,...,...,...
2953,More than 1000,27.0,male,Possibly,India,2019
2954,NaN,48.0,m,No,United States of America,2019
2955,NaN,50.0,M,No,India,2019
2956,More than 1000,30.0,female,Possibly,India,2019
